In [12]:
#-*- coding : utf-8 -*-
# coding: utf-8

import pandas as pd
from ipywidgets import interact
import json
import glob
import os

csv_df = None
category_mapping = {}

def loadJson():
    path = 'data/US_category_id.json'
    global category_mapping
    with open(path, 'r') as obj:
        json_data = json.load(obj)
        for item in json_data['items']:
            id = int(item['id'])
            if(id not in category_mapping):
                category_mapping[id] = item['snippet']['title']

def draw():
    print('draw...')

def show_top_n(properties, top_n, sort):
    print(csv_df.sort_values(properties, ascending = sort)[[properties]].head(top_n))
    draw()

def remove_duplicates():
    csv_df.drop_duplicates(subset='video_id', keep='first', inplace=True)
    csv_df.info()
    print('total size: ' + str(len(csv_df.index)))
    
def loadData(country):
    global csv_df
    if country=='ALL':
        return False
#         path_domestic = os.path.abspath(os.getcwd()) + '/data'
#         non_domestic = glob.glob(os.path.join(path_domestic, '*.csv'))
#         csv_df = pd.concat((pd.read_csv(f, encoding = 'utf-8', sep=',', error_bad_lines=False) for f in non_domestic))
#         csv_df = csv_df.set_index('title')
#         csv_df.info()
#         remove_duplicates()
    else:
        
        path = 'data/' + country + 'videos.csv'
        csv_df = pd.read_csv(path, encoding = 'utf-8', sep=',', error_bad_lines=False)
        csv_df = csv_df.set_index('title')
        csv_df.info()
        print('total size: ' + str(len(csv_df.index)))
        print('\n========================================================================================================\n')
        remove_duplicates()
        print('\n========================================================================================================\n')

loadJson()

interact(loadData, country=dict(美國="US", 日本="JP", 加拿大='CA', 德國='DE', 法國='FR',
                                英國='GB', 印度='IN', 墨西哥='MX', 南韓='KR', 俄羅斯='RU', 全部='ALL'))

interact(show_top_n, properties=dict(觀看次數='views', 喜歡='likes', 不喜歡='dislikes', 留言數量='comment_count'),
         top_n=(1,20), sort=dict(遞增=False, 遞減=True))

interactive(children=(Dropdown(description='country', options={'美國': 'US', '日本': 'JP', '加拿大': 'CA', '德國': 'DE'…

interactive(children=(Dropdown(description='properties', options={'觀看次數': 'views', '喜歡': 'likes', '不喜歡': 'disl…

<function __main__.show_top_n(properties, top_n, sort)>

In [2]:
categorys = {}

def findCategoryById(id):
    global categorys
    global category_mapping
    title = category_mapping[id]
    if title in categorys:
        categorys[title] = categorys[title]+1
    else:
        categorys[title] = 1

def findCategorys(country):
    global categorys
    categorys = {}
    path = 'data/' + country + 'videos.csv'
    data_df = pd.read_csv(path, encoding = 'utf-8', sep=',', error_bad_lines=False)
    data_df2 = data_df.drop_duplicates(subset='video_id', keep='first')
    for index, row in data_df2.iterrows():
        findCategoryById(row['category_id'])
    print('統計數量: ' + str(len(data_df2.index)))
    print(pd.DataFrame(list(sorted(categorys.items(), key=lambda x:x[1], reverse=True)),columns=['category', 'total']))

interact(findCategorys, country=dict(美國="US", 日本="JP", 加拿大='CA', 德國='DE', 法國='FR',
                                英國='GB', 印度='IN', 墨西哥='MX', 南韓='KR', 俄羅斯='RU'))


interactive(children=(Dropdown(description='country', options={'美國': 'US', '日本': 'JP', '加拿大': 'CA', '德國': 'DE'…

<function __main__.findCategorys(country)>

In [3]:
authors = {}
data_df2 = None

def show_result(top_n):
    print('統計數量: ' + str(len(data_df2.index)))
    print(pd.DataFrame(list(sorted(authors.items(), key=lambda x:x[1], reverse=True)),columns=['author', 'total']).head(top_n))

def findAuthors(country):
    global authors
    global data_df2
    path = 'data/' + country + 'videos.csv'
    data_df = pd.read_csv(path, encoding = 'utf-8', sep=',', error_bad_lines=False)
    data_df2 = data_df.drop_duplicates(subset='video_id', keep='first')
    authors = {}
    for index, row in data_df2.iterrows():
        author = row['channel_title']
        if author in authors:
            authors[author] = authors[author]+1
        else:
            authors[author] = 1
    interact(show_result, top_n=(1,20))
    
    
interact(findAuthors, country=dict(美國="US", 日本="JP", 加拿大='CA', 德國='DE', 法國='FR',
                                英國='GB', 印度='IN', 墨西哥='MX', 南韓='KR', 俄羅斯='RU'))


interactive(children=(Dropdown(description='country', options={'美國': 'US', '日本': 'JP', '加拿大': 'CA', '德國': 'DE'…

<function __main__.findAuthors(country)>